In [ ]:
!pip install pandarallel
from pandarallel import pandarallel

import pandas as pd
import csv

import numpy as np
from sklearn.model_selection import StratifiedKFold

from google.colab import drive
drive.mount('/content/drive')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction import text
from sklearn.pipeline import FeatureUnion,Pipeline
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier


%cd /content/drive/My\ Drive/SNA2020/Project

df_bots_pymorphy_badwords=pd.read_csv('./bots_pymorphy_badwords.tsv',sep="\t",header=0)
df_politics_pymorphy_badwords=pd.read_csv('./politics_pymorphy_badwords.tsv',sep="\t",header=0)
df_bots_politics_pymophy_badwords=df_bots_pymorphy_badwords.append(df_politics_pymorphy_badwords, ignore_index = True)
df_bots_politics_pymophy_badwords=df_bots_politics_pymophy_badwords[['pymorphy_badwords', 'reg_date', 'com_time_after', 'like_count', 'self_like', 'answer_count', 'group_id', 'has_media', 'Y']]

print("read_and_append")

# word_vectorizer = text.TfidfVectorizer(
#           analyzer='word', ngram_range=(1, 3),
#           min_df=2, use_idf=True, sublinear_tf=True)
# char_vectorizer = text.TfidfVectorizer(
#           analyzer='char', ngram_range=(3, 5),
#           min_df=2, use_idf=True, sublinear_tf=True)
# ngrams_vectorizer = Pipeline([('feats', FeatureUnion([('word_ngram', word_vectorizer),
#                 ('char_ngram', char_vectorizer),
#                 ])),])

# text_feature=ngrams_vectorizer.fit_transform(df_bots_politics_pymophy_badwords['pymorphy_badwords'].values)

vectorizer = text.TfidfVectorizer()
text_feature = vectorizer.fit_transform(df_bots_politics_pymophy_badwords['pymorphy_badwords'].values)

print("text_feature")
df_text_feature=pd.DataFrame(text_feature.toarray())
print("data_frame_text_feature")
df_text_feature['reg_date']=df_bots_politics_pymophy_badwords['reg_date']
df_text_feature['com_time_after']=df_bots_politics_pymophy_badwords['com_time_after']
df_text_feature['like_count']=df_bots_politics_pymophy_badwords['like_count']
df_text_feature['self_like']=df_bots_politics_pymophy_badwords['self_like']
df_text_feature['answer_count']=df_bots_politics_pymophy_badwords['answer_count']
df_text_feature['group_id']=df_bots_politics_pymophy_badwords['group_id']
df_text_feature['has_media']=df_bots_politics_pymophy_badwords['has_media']
y=df_bots_politics_pymophy_badwords['Y'].to_numpy()

print(df_text_feature)

X = df_text_feature.to_numpy()

skf = StratifiedKFold()
clf=LinearSVC(max_iter=10000)
pred = np.array([])

for train_index, test_index in skf.split(X, y):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    pred = np.append(pred, y_pred)
    print(len(pred))

print(classification_report(y,pred))

clf=RandomForestClassifier()
pred = np.array([])

for train_index, test_index in skf.split(X, y):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    pred = np.append(pred, y_pred)
    print(len(pred))

clf=AdaBoostClassifier(n_estimators=100, random_state=0)
pred = np.array([])

for train_index, test_index in skf.split(X, y):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    pred = np.append(pred, y_pred)
    print(len(pred))
# df_text_feature
# df_bots_politics_pymophy_badwords=pd.concat([df_text_feature, df_bots_politics_pymophy_badwords], axis=1)
# df_bots_politics_pymophy_badwords=df_text_feature.join(df_bots_politics_pymophy_badwords)
# df_bots_politics_pymophy_badwords = df_text_feature

# df_bots_politics_pymophy_badwords
# df_bots_politics_pymophy_badwords.to_csv('all_pymorphy.tsv',sep='\t', quoting=csv.QUOTE_NONE)


# df_bots_pymorphy_badwords.head()
# df_bots_pymorphy_badwords[10:20]
# df1.append(df2, ignore_index = True)
